In [1]:
# Enter your name here: DongJoo Chong
# Enter your Student ID number: G01093308

In [1]:
# Run cell to import libraries 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily 
import mapclassify 
import folium 
import fsspec

# This loads geodataframe contaning county geometry shapes
c = "https://github.com/babdelfa/gis/blob/main/counties_geometry.zip?raw=true"
import fsspec
with fsspec.open(c) as file:
    county_shapes = gpd.read_file(file)

# This loads the most recent covid19 data from Johns Hopkins University's Github
url_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
df_cases = pd.read_csv(url_cases) 
url_deaths = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df_deaths = pd.read_csv(url_deaths) 


#### FINAL CODE CONSOLIDATED  <br>
Except for the code above, include all your code below inside the single code cell. Run the code cell to display the output. <br>
Do **not** include additional cells.

In [ ]:
# Project code & output
df_drop = df_cases.drop(["UID", "iso2", "iso3", "code3", "FIPS", "Country_Region","Lat", "Long_"],axis=1).copy()
df_drop.rename(columns={"Admin2": "county", "Province_State": "state", "Combined_Key": "county_state"}, inplace=True)
df_cases_melted = pd.melt(df_drop, id_vars=["county","state","county_state"], var_name="dates", value_name="cases")
df_cases_melted.dates = pd.to_datetime(df_cases_melted.dates)
df1 = df_cases_melted.groupby(['dates'], as_index=False).sum()
df_date= df1.dates.max()

df = df_cases[['UID', 'iso2', 'FIPS', 'Admin2', 'Province_State', 'Lat', 'Long_', df_cases.columns[-1]]].copy()
df_d = df_deaths[['UID', 'iso2', 'FIPS', 'Admin2', 'Province_State', 'Lat', 'Long_', df_deaths.columns[-1]]].copy()
df.rename(columns={df.columns[-1] : "cases"}, inplace=True)
df_d.rename(columns={df_d.columns[-1] : "cases_d"}, inplace=True)
df_merged = pd.merge(df, df_d, how="left")

gdf = pd.merge(county_shapes, df_merged, left_on="FIPS_BEA", right_on="FIPS")
df_merged.rename(columns={"Admin2" : "County", "cases" : "Total Cases", "cases_d" : "Total Deaths"}, inplace =True)
Map_geo = gdf.copy()
Map_geo.rename(columns={"Admin2" : "County", "cases" : "Total Cases", "cases_d" : "Total Deaths"}, inplace =True)
Map_geo = Map_geo[["County", "Total Cases", "Total Deaths", "Province_State", "geometry"]].copy()
Map_geo.explore(column="Total Deaths", cmap="Set2", legend=True, scheme="natural_breaks")

# function to get the user information
def greeting():
    # Get the user name
    username = input("Enter your Name:")
    
    # Get the total cases number
    US_total_cases = df_merged['Total Cases'].sum()
    
    #  Get the total deaths number
    US_total_deaths = df_merged['Total Deaths'].sum()
    
    d1 = df_date.strftime("%m/%d/%y")
    print(f"Hello {username}")
    print("")
    print("MIS 433 Project")
    print(f"By: {username}")
    print(f"GMU ID: G01093308")
    print("")
    print("General data information:")
    print("        Topic: COVID-19 reported cases & death")
    print("        Data sources: Johns Hopkins University")
    print(f"        Time period: {d1}")
    print("")
    print("USA OVERVIEW")
    print(f"        - Total reported cases: {US_total_cases}")
    print(f"        - Total reported deaths: {US_total_deaths}")
    print("")
    print(f"{username}, enter a state name for more COVID-19 information:")

greeting()

def pickstate():
    global selected_state
# Get the state input
selected_state = input(">>>>>>Enter a state:")
state_match= df_merged['Province_State']

# Get the total cases and deaths number with user's input
total_cases = df_merged[df_merged['Province_State'] == selected_state]['Total Cases'].sum()
total_deaths = df_merged[df_merged['Province_State'] == selected_state]['Total Deaths'].sum()

Matched = 0
for x in state_match.index:
    if state_match[x].lower() == selected_state.lower():
        Matched = 1
if Matched == 1:
    print(f"Found information for {selected_state}.")
    print("")
    print("")
    print(f"{selected_state.upper()} OVERVIEW")
    print(f"        - Total reported cases: {total_cases}")
    print(f"        - Total reported deaths: {total_deaths}")
    print("")
    print("")
    print(f"Below is an interactive map of COVID-19 cases and deaths for {selected_state} counties.")
else:
    print("Cannot find a match")
    
pickstate()

# Show interactive Map
Map_f = Map_geo[Map_geo.Province_State==selected_state]
Map_f2 = Map_f[["County", "Total Cases", "Total Deaths", "geometry"]].copy()
Map_f2.explore(column="Total Deaths", cmap="Set2", legend=True, scheme="natural_breaks")

In [ ]:
merged_url_cases= url_cases.melt(column="Total Deaths", cmap="Set2", legend=True, scheme = "natural_breaks")
merged_url_cases = merged_url_cases[merged_url_cases['Province_states'] == selected_state]['Total Cases'].sum()